In [6]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install evaluate


In [8]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import is_bfloat16_supported
from evaluate import load

In [15]:
# Configuration
dtype = None  # Automatically detect dtype
load_in_4bit = True
max_seq_length = 2048
EOS_TOKEN = "<|endoftext|>"


In [ ]:
# Load the pre-trained model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/mistral-7b-v0.3",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

==((====))==  Unsloth 2025.1.6: Fast Mistral patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
# Prepare the LoRA model
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

In [11]:
# Load the dataset
dataset = load_dataset("Nassira/MedAnalyzr", split="train")

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

dataset (1).json:   0%|          | 0.00/2.32M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2182 [00:00<?, ? examples/s]

In [13]:
# Custom prompt template
alpaca_prompt = """Vous êtes un médecin professionnel spécialisé dans l'analyse des résultats de laboratoire médical pour évaluer la santé des patients. Examinez les valeurs de test fournies, comparez-les aux plages de référence normales, et déterminez si elles sont normales ou anormales. Fournissez une évaluation diagnostique basée sur les anomalies. Priorisez toujours la précision et le professionnalisme dans votre réponse.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


In [16]:
# Format the dataset
def formatting_prompts_func(examples):
    texts = []
    for instruction, input, output in zip(examples["instruction"], examples["input"], examples["output"]):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)


Map:   0%|          | 0/2182 [00:00<?, ? examples/s]

In [9]:
# Evaluation function
bleu = load("bleu")

def evaluate_model(dataset, model, tokenizer, sample_size=100):
    predictions, references = [], []
    dataset_sample = dataset.select(range(min(sample_size, len(dataset))))  # Limit to sample size

    for example in dataset_sample:
        inputs = tokenizer(
            alpaca_prompt.format(example["instruction"], example["input"], ""),
            return_tensors="pt"
        ).to("cuda")
        outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)
        decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

        # Extract the response part
        response_start = decoded_output.find("### Response:")
        if response_start != -1:
            prediction = decoded_output[response_start + len("### Response:"):].strip()
            predictions.append(prediction)

        references.append(example["output"])

    # Compute BLEU score
    bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
    return bleu_score



In [ ]:
# Evaluate before fine-tuning
FastLanguageModel.for_inference(model)

print("Evaluating before fine-tuning...")
eval_score_before = evaluate_model(dataset, model, tokenizer)
print("Evaluation BLEU Score (Before):", eval_score_before)

Evaluating before fine-tuning...
Evaluation BLEU Score (Before): {'bleu': 0.0008757654464720409, 'precisions': [0.009679370840895343, 0.0008161599673536013, 0.0006194507536650836, 0.0004179728317659352], 'brevity_penalty': 0.7323075163465832, 'length_ratio': 0.7624538745387454, 'translation_length': 4959, 'reference_length': 6504}


# Résultats de l'évaluation avant le fine-tuning

## BLEU Score (Avant Fine-Tuning)

### Score BLEU global
0.0009 (très faible, suggérant que le modèle n'est pas aligné sur la tâche spécifique)

### Précisions pour n-grammes
- **1-gramme** : 0.97%  
- **2-grammes** : 0.08%  
- **3-grammes** : 0.06%  
- **4-grammes** : 0.04%  
*(Les faibles précisions montrent que le modèle génère peu de correspondances avec les références, même pour des séquences courtes.)*

### Brevity Penalty (Pénalité de brièveté)
0.73  
*(Cela indique que les réponses générées par le modèle sont significativement plus courtes que les références attendues, avec un ratio longueur génération/référence de 0.76.)*

## Interprétation

Ces résultats montrent que le modèle pré-entraîné n'est pas encore adapté à la tâche spécifique (interprétation des analyses médicales dans ce cas).  
Le faible score BLEU et les précisions n-grammes basses reflètent le besoin d'un fine-tuning pour améliorer ses performances.


In [ ]:
# Fine-tuning
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=1,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)


Map:   0%|          | 0/2182 [00:00<?, ? examples/s]

In [ ]:
trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,182 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 272
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.996700
2,1.869200
3,1.865400
4,1.840000
5,1.905300
6,1.675900
7,1.688000
8,1.670800
9,1.825500
10,2.051400


TrainOutput(global_step=272, training_loss=1.7835772931575775, metrics={'train_runtime': 1371.1645, 'train_samples_per_second': 1.591, 'train_steps_per_second': 0.198, 'total_flos': 2.73728861294592e+16, 'train_loss': 1.7835772931575775, 'epoch': 0.997250229147571})

In [17]:
# Evaluate after fine-tuning
FastLanguageModel.for_inference(model)

print("Evaluating after fine-tuning...")
eval_score_before = evaluate_model(dataset, model, tokenizer)
print("Evaluation BLEU Score (Before):", eval_score_before)

Evaluating after fine-tuning...
Evaluation BLEU Score (Before): {'bleu': 0.05112201157334809, 'precisions': [0.2105323940408823, 0.0639093352027106, 0.030972928242108996, 0.01638952027754516], 'brevity_penalty': 1.0, 'length_ratio': 1.3313345633456335, 'translation_length': 8659, 'reference_length': 6504}


# 🌟 **Analyse des Résultats Avant et Après le Fine-Tuning** 🌟

---

## 📊 **Résultats Avant Fine-Tuning**

### 🔹 **Score BLEU Global**
- **Avant** : **0.0009** (très faible, le modèle n'est pas aligné sur la tâche spécifique)

### 🔹 **Précisions pour les n-grammes**
- **1-gramme** : **0.97%**  
- **2-grammes** : **0.08%**  
- **3-grammes** : **0.06%**  
- **4-grammes** : **0.04%**  
> 📝 *(Ces faibles précisions montrent que le modèle générait très peu de correspondances avec les références.)*

### 🔹 **Brevity Penalty (Pénalité de Brièveté)**
- **Avant** : **0.73**  
*(Les réponses générées étaient significativement plus courtes que les références)*

### 📌 **Interprétation Avant Fine-Tuning**
- Le modèle pré-entraîné n'était pas adapté à la tâche spécifique (interprétation des analyses médicales).
- Les résultats faibles reflétaient le besoin urgent d'un fine-tuning pour améliorer les performances.

---

## 📊 **Résultats Après Fine-Tuning**

### 🔹 **Score BLEU Global**
- **Après** : **0.0511**  *(≈50x plus élevé qu'avant, indiquant une nette amélioration)*

### 🔹 **Précisions pour les n-grammes**
- **1-gramme** : **21.05%**  *(grande amélioration sur les mots simples)*
- **2-grammes** : **6.39%**  *(amélioration pour les séquences de deux mots)*
- **3-grammes** : **3.09%**  *(progrès pour des séquences plus longues)*
- **4-grammes** : **1.63%**  *(encore faible, mais mieux qu'avant)*

### 🔹 **Brevity Penalty (Pénalité de Brièveté)**
- **Après** : **1.0**  *(aucune pénalité, les réponses sont désormais adaptées en longueur)*
- **Ratio Longueur Génération/Référence** : **1.33**  *(les réponses générées sont un peu plus longues, mais dans des limites acceptables)*

### 📌 **Interprétation des Résultats Après Fine-Tuning**
- Le modèle capture désormais mieux les correspondances avec les références.
- Les précisions n-grammes montrent des améliorations significatives à tous les niveaux.
- La longueur des réponses est désormais optimale, sans pénalité de brièveté.

---

## ⚠️ **Points à Améliorer**
- Les scores pour les **3-grammes** et **4-grammes** restent faibles, ce qui indique des difficultés à produire des séquences plus complexes et contextuelles.
- Le score BLEU global, bien qu'amélioré, est encore loin d'être excellent.

---

## 🚀 **Recommandations pour des Améliorations Supplémentaires**

1. **Entraîner le modèle sur un dataset plus large et diversifié** pour améliorer les correspondances n-grammes.
2. **Ajuster les hyperparamètres** *(learning rate, batch size)* pour un fine-tuning plus précis.
3. **Ajouter des techniques avancées** comme :
   - **Data Augmentation** pour enrichir les données.
   - **Curriculum Learning** pour entraîner le modèle progressivement sur des tâches plus complexes.
4. **Explorer des architectures supplémentaires** ou des pertes spécifiques :
   - Objectif multitâche
   - Alignement avec des métriques de génération avancées.

---

✨ *Ces résultats montrent une amélioration claire grâce au fine-tuning, mais il reste des pistes intéressantes à explorer pour des performances optimales.*
